In [85]:
%reset

import os
import pandas
import json
import re #remove_tags function
os.chdir('C:\\python 3.8\\app0')
os.getcwd()

#+ function
##  remove tag
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [86]:
filedir = 'data\\data-1588062239620.csv'
filedir_all = os.getcwd() + '\\' + filedir
print(filedir_all)

data = pandas.read_csv(filedir_all)

data = pandas.DataFrame(data)

C:\python 3.8\app0\data\data-1588062239620.csv


In [87]:
print(data.shape)
print(data['cname'].nunique())
#print(data.groupby('cname').count())
stat_cname = data.groupby('cname').count()
stat_cname.to_csv('data\\full_stat\\stat_cname.csv', sep = ';', encoding='utf-8-sig')

(24153, 57)
76


In [88]:
#convert ['q_update_ts'] to time#
#################################
data['q_update_ts'] = pandas.to_datetime(data['q_update_ts'])

#stat
print('max ' + data['update_ts'].max())
print('min ' + data['update_ts'].min())

max 2020-03-12 07:58:50.445738
min 2015-04-13 19:51:47.225


In [89]:
#pure text

data['text'] = data['prompt'].apply(lambda x:
                                    remove_tags(json.loads(x)['definition']['value']))

In [90]:
## create group for future aggregate

#position id
grouped = data.groupby(['cname','bid','course_id',
                        'm_ord', 'l_ord', 'i_ord', 'subi_ord', 'aq_ord'])

#position - aq_ord + internal_id ##'i_ord', 'subi_ord',
grouped_ext_id = data.groupby(['cname','bid','course_id',
                               'm_ord', 'l_ord',  'internal_id'])

In [91]:
#tab0 OR статистика posit id
tab0 = grouped.agg(question_ext_id = ('question_ext_id', 'nunique'),
                   text = ('text', 'nunique'),
                   internal_id = ('internal_id', 'nunique'),
                   cnt_pos = ('cnt', 'sum')
                  ).reset_index()

tab0.to_csv('data\\full_stat\\tab0.csv', sep = ';', encoding='utf-8', index=False)

In [92]:
#sum of cnt
#maximum time of change 'update_ts'
res = grouped_ext_id.agg(question_ext_id = ('question_ext_id', 'nunique'),
                         UNIQUEtext = ('text', 'nunique'),
                         tot_num = ('tot_num', 'sum'),
                         cnt = ('cnt', 'sum'),
                         update_ts = ('update_ts', 'max')                         
                        ).reset_index()

#stat!
print('res.shape -->')
print(res.shape)
#print('res.columns -->')
#print(res.columns)
#print(res[res.m_ord == 4])

res.shape -->
(9666, 11)


In [93]:
#create posit for res ##'i_ord', 'subi_ord', 
column_for_posit = ['cname','bid','course_id', 'm_ord', 'l_ord', 'internal_id', 'update_ts']
#
res['posit'] = res[column_for_posit].astype(str).apply('|'.join,1)
#print('res.columns -->')
#res.columns

In [94]:
#create posit for original -не совсем position
data['posit'] = data[column_for_posit].astype(str).apply('|'.join,1)

#data124.head
data_pos_text = data[['posit', 'text']]
print(data_pos_text.shape)

#drop_duplicates
data_pos_text = data_pos_text.drop_duplicates(subset = 'posit') 
print(data_pos_text.shape)

(24153, 2)
(12428, 2)


In [95]:
#merge two data frames
print('res.shape -->')
print(res.shape)

result = pandas.merge(res, data_pos_text, how='left', on='posit', indicator=False)

print('result.shape -->')
print(result.shape)
print('result.columns -->')
print(result.columns)

#result

res.shape -->
(9666, 12)
result.shape -->
(9666, 13)
result.columns -->
Index(['cname', 'bid', 'course_id', 'm_ord', 'l_ord', 'internal_id',
       'question_ext_id', 'UNIQUEtext', 'tot_num', 'cnt', 'update_ts', 'posit',
       'text'],
      dtype='object')


In [96]:
#ord data by l + i + subi + aq_ord + cnt ##- 'i_ord', 'subi_ord',
ord_columns = ['cname','bid','course_id', 'm_ord', 'l_ord',  'cnt']
result = result.sort_values(by=ord_columns)

In [97]:
#print(result.dtypes) ##, 'i_ord', 'subi_ord'
column_for_posit = [ 'm_ord', 'l_ord'] #for ranking
result['pos_for_rank'] = result[column_for_posit].astype(str).apply(''.join,1)

# convert column 'pos_for_rank' to numeric
result['pos_for_rank_num'] = pandas.to_numeric(result['pos_for_rank'])


result['x_pos'] = result.groupby(['cname','bid','course_id','m_ord'])['pos_for_rank_num'].rank(method='first').astype(int)
result['y_pos'] = result['m_ord'] + 1

needed_col = ['cname','bid','course_id', 'm_ord', 'l_ord', 'y_pos',  'x_pos', 'internal_id', 'question_ext_id', 'UNIQUEtext', 'update_ts', 'tot_num','cnt',  'text'] #
result = result[needed_col]

result.to_csv('data\\full_stat\\tab1.csv', sep = ';', encoding='utf-8-sig', index=False)

